In [1]:
# this file aims at using transfer learning, to get the result in somewhat middle part of the computational
# graph, this time I use inception_resnet_v2
import tensorflow as tf
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from data_preprocessing import image_preprocessing
slim = tf.contrib.slim

In [2]:
checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'
label_file = 'training/label.idl'
train_file = 'training/'
validation_file = 'validation/'

In [3]:
def get_loss(box_confidence, confidence, box_coordinate, coordinate, box_class_label, class_label):
    # first get all the loss which grids have target in it
    filtering_mask_obj = tf.reshape(box_confidence > 0.5, [-1, 18, 32])
    
    box_confidence_obj = tf.boolean_mask(box_confidence, filtering_mask_obj)
    confidence_obj = tf.boolean_mask(confidence, filtering_mask_obj)
    box_coordinate_obj = tf.boolean_mask(box_coordinate, filtering_mask_obj)
    coordinate_obj = tf.boolean_mask(coordinate, filtering_mask_obj)
    box_class_label_obj = tf.boolean_mask(box_class_label, filtering_mask_obj)
    class_label_ojb = tf.boolean_mask(box_class_label, filtering_mask_obj)
    
    coor_loss = 5 * tf.reduce_sum(tf.square(tf.subtract(box_coordinate_obj, coordinate_obj)))
    conf_loss = tf.reduce_sum(tf.square(tf.subtract(box_confidence_obj, confidence_obj)))
    class_loss = tf.reduce_sum(tf.square(tf.subtract(box_class_label_obj, class_label_ojb)))
    
    # get all the loss which grids have no target in it
    filtering_mask_noobj = tf.reshape(box_confidence < 0.5, [-1, 18, 32])
    
    box_confidence_noobj = tf.boolean_mask(box_confidence, filtering_mask_noobj)
    confidence_noobj = tf.boolean_mask(confidence, filtering_mask_noobj)
    
    conf_loss_noobj = 0.5 * tf.reduce_sum(tf.square(tf.subtract(box_confidence_noobj, confidence_noobj)))
    
    loss = coor_loss + conf_loss + class_loss + conf_loss_noobj
    
    return loss

In [4]:
# build the computational graph from inception_resnet_v2 and extract a mid point.

images = tf.placeholder(tf.float32, shape=(None, 360, 640, 3))
box_confidence = tf.placeholder(tf.float32, shape=(None, 18, 32, 1))
box_coordinate = tf.placeholder(tf.float32, shape=(None, 18, 32, 4))
box_class_label = tf.placeholder(tf.float32, shape=(None, 18, 32, 4))

images_norm = tf.divide(images, 255)
with slim.arg_scope(inception_resnet_v2_arg_scope()):
    _, end_points = inception_resnet_v2(images_norm, num_classes = 1001, is_training = False)    

Mixed_5b = end_points['Mixed_5b']


In [5]:
with tf.name_scope('my_fine_tune') as scope:
    conv1 = tf.layers.conv2d(Mixed_5b, filters=160, kernel_size=(3,3), strides=2, padding = 'valid',
                             activation = tf.nn.relu, name='conv1')
#     weight1= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
    batch_norm1 = tf.layers.batch_normalization(conv1, axis = -1)
    
    conv2 = tf.layers.conv2d(inputs=batch_norm1, filters=80, kernel_size=(1,3), strides=1, 
                                padding = 'VALID', activation = tf.nn.relu, name='conv2')
    batch_norm2 = tf.layers.batch_normalization(conv2, axis = -1)
    
    conv3 = tf.layers.conv2d(inputs=batch_norm2, filters = 9, kernel_size=(1,3), strides=1,
                            padding = 'VALID', activation = tf.nn.relu, name='conv3')
    batch_norm3 = tf.layers.batch_normalization(conv3, axis = -1)
    
    confidence = tf.layers.conv2d(inputs=batch_norm3, filters = 1, kernel_size=(3,3), strides=1,
                            padding = 'VALID', activation = tf.nn.relu, name='conv_confidence')
    coordinate = tf.layers.conv2d(inputs=batch_norm3, filters = 4, kernel_size=(3,3), strides=1,
                            padding = 'VALID', activation = tf.nn.relu, name='conv_coordinate')
    class_label = tf.layers.conv2d(inputs=batch_norm3, filters = 4, kernel_size=(3,3), strides=1,
                            padding = 'VALID', activation = tf.nn.relu, name='conv_class_label')
    
    loss = get_loss(box_confidence, confidence, box_coordinate, coordinate, box_class_label, class_label)
    
    # define the variables that I want to train
    weight1= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
    bias1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[1]
    
    weight2= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[0]
    bias2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[1]
    
    weight3= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[0]
    bias3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[1]
    
    weight_confidence= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_confidence')[0]
    bias_confidence = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_confidence')[1]
    
    weight_coordinate= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_coordinate')[0]
    bias_coordinate = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_coordinate')[1]
    
    weight_class_label= tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_class_label')[0]
    bias_class_label = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv_class_label')[1]
    
    variables_to_train = [weight1,weight2,weight3,weight_confidence,weight_coordinate,weight_class_label,
                         bias1,bias2,bias3,bias_confidence,bias_coordinate,bias_class_label]
    
    # define the optimizer and train operation.
    optimizer = tf.train.AdamOptimizer()
    train_op = slim.learning.create_train_op(loss, optimizer, variables_to_train=variables_to_train)
    
# define the variables that I don't need to restore
exclude = ['InceptionResnetV2/AuxLogits', 'InceptionResnetV2/Logits','my_fine_tune/',
          'conv1','conv2','conv3','conv_confidence','conv_coordinate','conv_class_label',
          'batch_normalization','batch_normalization_2','batch_normalization_3']

variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
saver = tf.train.Saver(variables_to_restore)
init = tf.global_variables_initializer()

/Users/zhengyixing/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [8]:
image_preprocessor = image_preprocessing(train_file, label_file, validation_file)
image_preprocessor.prepare_data()

In [10]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, checkpoint_file)
    for i in range(100):
        the_images, labels = image_preprocessor.get_next_image_batch()
        the_loss = sess.run(train_op, feed_dict={images: the_images, 
                                                 box_confidence: labels['confidence'],
                                                 box_class_label: labels['class_label'],
                                                 box_coordinate: labels['coordinate']})
        
        if(i % 10 == 0):
            validation_loss = sess.run(loss, feed_dict={images:image_preprocessor.validation_images,
                                                        box_confidence: image_preprocessor.validation_labels['confidence'],
                                                        box_class_label: image_preprocessor.validation_labels['class_label'],
                                                        box_coordinate: image_preprocessor.validation_labels['coordinate']})

            print('###############################################################')
            print('the loss after ' + str(i) + ' iteration is: ' + str(the_loss))
            print('the validation loss after ' + str(i) + ' iteration is: ' + str(validation_loss))
        

INFO:tensorflow:Restoring parameters from ./inception_resnet_v2_2016_08_30.ckpt
###############################################################
the loss after 0 iteration is: 1971.03
the validation loss after 0 iteration is: 50305.8
###############################################################
the loss after 1 iteration is: 6860.02
the validation loss after 1 iteration is: 29136.4
###############################################################
the loss after 2 iteration is: 4412.55
the validation loss after 2 iteration is: 14612.0
###############################################################
the loss after 3 iteration is: 2603.67
the validation loss after 3 iteration is: 15610.6
###############################################################
the loss after 4 iteration is: 2483.37
the validation loss after 4 iteration is: 15195.8
###############################################################
the loss after 5 iteration is: 1645.56
the validation loss after 5 iteration is: 15057.9
##

In [ ]:
image_preprocessor.labels['class_label'].shape

In [ ]:
heheh = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'batch_normalization_2')

In [ ]:
print(images)

In [ ]:
_, labels = image_preprocessor.get_next_image_batch()

In [ ]:
(labels['coordinate'].shape)